In [1]:
# ! pip install quandl

### Import Libraries

In [2]:
# This program predicts stock prices using machine learning models

import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split


### Downloading data

In [3]:
# we are downloading stock data for SBI 
df=quandl.get("NSE/SBIN")

In [4]:
df.columns

Index(['Open', 'High', 'Low', 'Last', 'Close', 'Total Trade Quantity',
       'Turnover (Lacs)'],
      dtype='object')

In [5]:
df.head()

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
Date,,,,,,,
1998-03-20,275.40,278.80,273.30,NaN,NaN,NaN,NaN
1998-03-23,278.45,280.90,278.45,279.95,279.95,591700.0,1654.91
1998-03-24,282.00,288.40,280.00,282.25,284.35,4979900.0,14231.55
1998-03-25,285.00,290.25,284.50,288.70,289.15,3632600.0,10436.93
1998-03-26,289.00,289.00,280.10,281.75,281.70,3861750.0,10996.67


In [6]:
sbi=df

In [45]:
sbi['Close'].tail(30)

Date
2018-11-22    282.70
2018-11-26    286.40
2018-11-27    289.40
2018-11-28    284.55
2018-11-29    286.40
2018-11-30    284.65
2018-12-03    286.80
2018-12-04    282.55
2018-12-05    280.15
2018-12-06    275.65
2018-12-07    275.40
2018-12-10    274.20
2018-12-11    281.25
2018-12-12    285.25
2018-12-13    289.00
2018-12-14    289.20
2018-12-17    289.50
2018-12-18    292.75
2018-12-19    300.70
2018-12-20    294.05
2018-12-21    291.90
2018-12-24    293.05
2018-12-26    294.15
2018-12-27    292.15
2018-12-28    294.80
2018-12-31    295.90
2019-01-01    299.60
2019-01-02    293.90
2019-01-03    291.10
2019-01-04    297.65
Name: Close, dtype: float64

In [7]:
null_rows = df[df.isnull().any(axis=1)]
null_rows

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
Date,,,,,,,
1998-03-20,275.4,278.8,273.3,NaN,NaN,NaN,NaN


In [8]:
sbi.fillna(method='ffill', inplace=True) 
sbi.fillna(method='bfill', inplace=True) 

C:\Users\asus\AppData\Local\Temp\ipykernel_18708\2090128648.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sbi.fillna(method='ffill', inplace=True)
C:\Users\asus\AppData\Local\Temp\ipykernel_18708\2090128648.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sbi.fillna(method='bfill', inplace=True)


In [9]:
sbi

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
Date,,,,,,,
1998-03-20,275.40,278.80,273.30,279.95,279.95,591700.0,1654.91
1998-03-23,278.45,280.90,278.45,279.95,279.95,591700.0,1654.91
1998-03-24,282.00,288.40,280.00,282.25,284.35,4979900.0,14231.55
1998-03-25,285.00,290.25,284.50,288.70,289.15,3632600.0,10436.93
1998-03-26,289.00,289.00,280.10,281.75,281.70,3861750.0,10996.67
...,...,...,...,...,...,...,...
2018-12-31,297.20,298.25,295.50,295.65,295.90,9526067.0,28244.43
2019-01-01,297.50,300.70,293.85,300.70,299.60,11837127.0,35128.77
2019-01-02,299.10,302.50,293.10,293.80,293.90,25559853.0,76220.10


In [10]:
sbi[sbi.isnull().any(axis=1)]

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
Date,,,,,,,


In [11]:
# Get adjusted close price

sbi_close=df[['Close']]
sbi_close


,Close
Date,
1998-03-20,279.95
1998-03-23,279.95
1998-03-24,284.35
1998-03-25,289.15
1998-03-26,281.70
...,...
2018-12-31,295.90
2019-01-01,299.60
2019-01-02,293.90


In [19]:
# a variable for predicting 'n' days out into the future

forecast_out =30

In [20]:
sbi_close['Predicton']=sbi[['Close']].shift(-forecast_out)

C:\Users\asus\AppData\Local\Temp\ipykernel_18708\443889611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sbi_close['Predicton']=sbi[['Close']].shift(-forecast_out)


In [21]:
sbi_close.head()

,Close,Predicton
Date,,
1998-03-20,279.95,282.05
1998-03-23,279.95,278.05
1998-03-24,284.35,278.60
1998-03-25,289.15,263.95
1998-03-26,281.70,256.60


In [22]:
sbi_close.tail()

,Close,Predicton
Date,,
2018-12-31,295.90,NaN
2019-01-01,299.60,NaN
2019-01-02,293.90,NaN
2019-01-03,291.10,NaN
2019-01-04,297.65,NaN


In [25]:
X=np.array(sbi_close.drop(['Predicton'],axis=1))
X=X[:-forecast_out]
X

array([[279.95],
       [279.95],
       [284.35],
       ...,
       [288.15],
       [283.45],
       [287.05]])

In [29]:
# Create dependent data set
# Convert a dataframe to numpy array
Y=np.array(sbi_close['Predicton'])
Y=Y[:-forecast_out]
Y


array([282.05, 278.05, 278.6 , ..., 293.9 , 291.1 , 297.65])

In [30]:
# split the data into 80% training and 20% testing 
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [31]:
svr_rbf=SVR(kernel='rbf',C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, gamma=0.1)

In [37]:
svr_confidence=svr_rbf.score(x_test,y_test)
print("SVM confidence:",svr_confidence)


SVM confidence: 0.8944588492885903


In [36]:
# Create and train the Linear Regression Model
lr=LinearRegression()
#Train the model
lr.fit(x_train,y_train)

LinearRegression()

In [38]:
lr_confidence=lr.score(x_test,y_test)
print("LR confidence:",lr_confidence)


LR confidence: 0.9024580410378453


In [40]:
# Set forecast 
x_forecast=np.array(sbi_close.drop(['Predicton'],axis=1))[-forecast_out:]
x_forecast

array([[282.7 ],
       [286.4 ],
       [289.4 ],
       [284.55],
       [286.4 ],
       [284.65],
       [286.8 ],
       [282.55],
       [280.15],
       [275.65],
       [275.4 ],
       [274.2 ],
       [281.25],
       [285.25],
       [289.  ],
       [289.2 ],
       [289.5 ],
       [292.75],
       [300.7 ],
       [294.05],
       [291.9 ],
       [293.05],
       [294.15],
       [292.15],
       [294.8 ],
       [295.9 ],
       [299.6 ],
       [293.9 ],
       [291.1 ],
       [297.65]])

In [41]:
lr_prediction=lr.predict(x_forecast)
lr_prediction

array([309.71738375, 313.27729011, 316.16370066, 311.49733693,
       313.27729011, 311.59355062, 313.66214485, 309.57306323,
       307.26393478, 302.93431895, 302.69378474, 301.53922052,
       308.32228532, 312.17083273, 315.77884592, 315.97127329,
       316.25991434, 319.38685911, 327.03584708, 320.63763702,
       318.56904279, 319.67550017, 320.7338507 , 318.809577  ,
       321.35923966, 322.41759019, 325.97749654, 320.49331649,
       317.79933331, 324.10132968])

In [43]:
svr_predction=svr_rbf.predict(x_forecast)
svr_predction

array([288.88332093, 288.57886251, 284.49885763, 289.40721836,
       288.57886251, 288.78509923, 289.14998445, 287.23994942,
       279.95358033, 277.73364939, 275.11358243, 266.40465544,
       277.20338401, 286.38522101, 283.86507243, 284.16081394,
       284.64836142, 274.84293844, 280.73252214, 285.95779754,
       272.07944615, 277.74152949, 286.18046703, 271.83188381,
       284.09803862, 274.53153608, 282.73081532, 285.35013652,
       277.59097363, 279.91173303])